In [ ]:
import numpy as np
import pandas as pd
import random
import os
import time
from pathlib import Path

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.metrics import roc_auc_score

#import lightgbm as lgb
#import xgboost as xgb
import catboost as ctb

#import matplotlib.pyplot as plt
#import seaborn as sns

import warnings
warnings.simplefilter('ignore')

# Parameters

In [ ]:
target = 'claim'

DEBUG = False

if DEBUG:
    N_ESTIMATORS = 1
    N_SPLITS = 2
    SEED = 2017
    CVSEED = 2017
    EARLY_STOPPING_ROUNDS = 1
    VERBOSE = 100
    #N_ITERS = 2
else:
    N_SPLITS = 10
    N_ESTIMATORS = 20000
    EARLY_STOPPING_ROUNDS = 300
    VERBOSE = 1000
    SEED = 2017
    CVSEED = 2017
    #N_ITERS = 10

In [ ]:
def set_seed(seed=2017):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    
set_seed(SEED)

# Datasets

In [ ]:
INPUT = Path("../input/tabular-playground-series-sep-2021")

train = pd.read_csv(INPUT / "train.csv")
#test = pd.read_csv(INPUT / "test.csv")
#submission = pd.read_csv(INPUT / "sample_solution.csv")

# Preprocessing

In [ ]:
features = [col for col in train.columns if 'f' in col]

In [ ]:
train['n_missing'] = train[features].isna().sum(axis=1)
#test['n_missing'] = test[features].isna().sum(axis=1)

features += ['n_missing']

In [ ]:
train[features] = train[features].fillna(train[features].median())
#test[features] = test[features].fillna(test[features].median())

In [ ]:
ss = StandardScaler()


In [ ]:
train[features]

In [ ]:
train[target]

# Optuna

In [ ]:
# Optuna for parameter search
!pip install -q optuna

import optuna
import pickle

In [ ]:
def objective(trial, X=train[features], y=train[target]):
 
  param_space = {
                     'objective': 'CrossEntropy',
                     'n_estimators':N_ESTIMATORS,
                     'thread_count' : -1,
                     'bootstrap_type': 'Bernoulli',
                     'eval_metric': 'AUC',
           
               'learning_rate':trial.suggest_loguniform('learning_rate', 1e-4, 3),
               'subsample': trial.suggest_uniform('subsample', 0.3124, 0.5124),
               'colsample_bylevel':trial.suggest_uniform('colsample_bylevel', 0.2979, 0.4979),
               'max_depth':trial.suggest_int('max_depth', 5, 7),
               'min_child_samples':trial.suggest_int('min_child_samples', 5, 76),
               'max_bin':trial.suggest_int('max_bin', 290, 400),
               'reg_lambda':trial.suggest_loguniform('reg_lambda', 1e-4, 5),
               'random_strength': trial.suggest_uniform('random_strength', 34.23, 46.23),
      
               # 'device':'gpu',  # Use GPU acceleration
               # 'gpu_platform_id': 0,
               # 'gpu_device_id': 0,
                 #'booster': 'gbtree',
                  #'tree_method': 'hist',
                 #'importance_type': 'total_gain',
               # 'reg_alpha':trial.suggest_loguniform('reg_alpha', 1e-4, 50),
               #'min_child_weight':trial.suggest_uniform('min_child_weight', 1e-4, 50),
                   #'gamma': trial.suggest_uniform('gamma', 1e-4, 50)   
                #'num_leaves':trial.suggest_int('num_leaves', 50, 500),
                #'subsample_freq': trial.suggest_int('subsample_freq', 1, 10),
               #'metric':'auc',
              #'min_split_gain': trial.suggest_float('min_split_gain', 0.0, 0.005),
              #'class_weight':trial.suggest_categorical('class_weight',['balanced',None]),
              #'cat_smooth':trial.suggest_int('cat_smooth', 5, 100),
              #'cat_l2':trial.suggest_loguniform('cat_l2', 1e-3, 100)
                }
            

  seed_list=[SEED]
  kf = StratifiedKFold(n_splits=N_SPLITS, shuffle=True, random_state=CVSEED)
  #ctb_oof = np.zeros(train.shape[0])
  #ctb_pred = np.zeros(test.shape[0])
  
  #X_tr, X_va, y_tr, y_va = train_test_split(X,y,test_size=.2,random_state=CVSEED)
  
  for fold, (trn_idx, val_idx) in enumerate(kf.split(X, y)):
      print(f"===== fold {fold} =====")

      X_train = train[features].iloc[trn_idx]
      y_train = train[target].iloc[trn_idx]
      X_valid = train[features].iloc[val_idx]
      y_valid = train[target].iloc[val_idx]
      #X_test = test[features]

      X_train[features] = ss.fit_transform(X_train[features])
      X_valid[features] = ss.transform(X_valid[features])
      #X_test[features] = ss.transform(X_test[features])

      pred=np.zeros(y_valid.shape[0])

      start = time.time()
      for inseed in seed_list:
            param_space['random_state'] = inseed

            model = ctb.CatBoostClassifier(**param_space)
            model.fit(
                X_train, 
                y_train,
                eval_set=[(X_valid, y_valid)],
                use_best_model=True,
                early_stopping_rounds=EARLY_STOPPING_ROUNDS,
                verbose=VERBOSE,
            )

            #ctb_oof[val_idx] = model.predict_proba(X_valid)
            pred += model.predict_proba(X_valid)[:,-1] / len(seed_list)

      elapsed = time.time() - start
      auc = roc_auc_score(y_valid, pred)

      print(f"fold {fold} - ctb rmse: {auc:.6f}, elapsed time: {elapsed:.2f}sec\n")            

      #print(f"oof ctb_rmse = {mean_squared_error(train[target], ctb_oof, squared=False)}")
  
  return auc

In [ ]:
study = optuna.create_study(direction='maximize')
study.optimize(objective,n_trials= 30)
print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)

In [ ]:
study.best_params

# Log

0.8146581604430743 ver6 of ver 4-6

0.814972634808961 ver8 of ver7-9

no improvement ver10-12